In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertModel, TFBertForSequenceClassification
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, GRU, Dense, Dropout, Conv1D, GlobalMaxPooling1D
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

# PyTorch CUDA Bellek Yönetimi
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# GPU Kullanımını Kontrol Et
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Kullanılan cihaz: {device}")

# Temizlenmiş Veri Yükleme
print("Temizlenmiş veri yükleniyor...")
df_cleaned = pd.read_csv('/content/processed_text.csv', encoding='utf-8')

# Orijinal Veri Yükleme ve Label Kolonunu Ekleme
print("Orijinal veri yükleniyor...")
df_original = pd.read_csv('train_0.csv', encoding='utf-8')
df_cleaned['Label'] = df_original['Label']

# Label Kolonunu One-Hot Encode
print("Label sütunu one-hot encode ediliyor...")
one_hot_encoder = OneHotEncoder(sparse_output=False)
one_hot_labels = one_hot_encoder.fit_transform(df_cleaned['Label'].values.reshape(-1, 1))

# Eğitim ve Test Verisi Ayrımı
print("Veriler hazırlanıyor...")
X_train, X_test, y_train, y_test = train_test_split(df_cleaned['Processed_Text'], one_hot_labels, test_size=0.2, random_state=42)

# BERT Tokenizer ve Model Yükleme
print("BERT modeli ve tokenizer yükleniyor...")
bert_tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
bert_model = BertModel.from_pretrained("dbmdz/bert-base-turkish-cased").to(device)

# Tokenize ve Pad İşlemi
print("Veriler tokenize ediliyor...")
def tokenize_and_pad(texts, tokenizer, max_len=128):
    tokens = tokenizer(texts.tolist(), max_length=max_len, padding='max_length', truncation=True, return_tensors='pt')
    return tokens['input_ids'], tokens['attention_mask']

X_train_ids, X_train_mask = tokenize_and_pad(X_train, bert_tokenizer)
X_test_ids, X_test_mask = tokenize_and_pad(X_test, bert_tokenizer)

# Verileri GPU'ya Taşı
X_train_ids, X_train_mask = X_train_ids.to(device), X_train_mask.to(device)
X_test_ids, X_test_mask = X_test_ids.to(device), X_test_mask.to(device)

# BERT'ten Embedding Alma
print("BERT embedding'leri GPU üzerinde hesaplanıyor...")
def process_in_batches(input_ids, attention_mask, model, batch_size=32):
    embeddings = []
    for i in range(0, len(input_ids), batch_size):
        batch_input_ids = input_ids[i:i + batch_size]
        batch_attention_mask = attention_mask[i:i + batch_size]
        with torch.no_grad():
            outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
            embeddings.append(outputs.last_hidden_state[:, 0, :].cpu().numpy())
    return np.vstack(embeddings)

X_train_embeddings = process_in_batches(X_train_ids, X_train_mask, bert_model)
X_test_embeddings = process_in_batches(X_test_ids, X_test_mask, bert_model)

Kullanılan cihaz: cuda
Temizlenmiş veri yükleniyor...
Orijinal veri yükleniyor...
Label sütunu one-hot encode ediliyor...
Veriler hazırlanıyor...
BERT modeli ve tokenizer yükleniyor...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Veriler tokenize ediliyor...
BERT embedding'leri GPU üzerinde hesaplanıyor...


In [ ]:
# CNN Modeli
print("CNN modeli eğitiliyor...")
cnn_model = Sequential([
    Embedding(input_dim=len(bert_tokenizer.vocab), output_dim=128, input_length=128),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(one_hot_labels.shape[1], activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn_history = cnn_model.fit(X_train_embeddings, y_train, epochs=10, batch_size=16, validation_data=(X_test_embeddings, y_test))

# LSTM Modeli
print("LSTM modeli eğitiliyor...")
lstm_model = Sequential([
    Embedding(input_dim=len(bert_tokenizer.vocab), output_dim=128, input_length=128),
    Bidirectional(LSTM(64, return_sequences=True)),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(one_hot_labels.shape[1], activation='softmax')
])

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
lstm_history = lstm_model.fit(X_train_embeddings, y_train, epochs=10, batch_size=16, validation_data=(X_test_embeddings, y_test))

# GRU Modeli
print("GRU modeli eğitiliyor...")
gru_model = Sequential([
    Embedding(input_dim=len(bert_tokenizer.vocab), output_dim=128, input_length=128),
    Bidirectional(GRU(64, return_sequences=True)),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(one_hot_labels.shape[1], activation='softmax')
])

gru_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
gru_history = gru_model.fit(X_train_embeddings, y_train, epochs=10, batch_size=16, validation_data=(X_test_embeddings, y_test))




CNN modeli eğitiliyor...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.5222 - loss: 0.9929 - val_accuracy: 0.5464 - val_loss: 0.9521
Epoch 2/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.5296 - loss: 0.9780 - val_accuracy: 0.5467 - val_loss: 0.9364
Epoch 3/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.5281 - loss: 0.9684 - val_accuracy: 0.5464 - val_loss: 0.9310
Epoch 4/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.5306 - loss: 0.9632 - val_accuracy: 0.5475 - val_loss: 0.9209
Epoch 5/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5367 - loss: 0.9493 - val_accuracy: 0.5547 - val_loss: 0.9208
Epoch 6/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.5442 - loss: 0.9402 - val_accuracy: 0.5558 - val_loss: 0.9151
Epoch 7/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5360 - loss: 0.9429 - val_accuracy: 0.5625 - val_loss: 0.9168
Epoch 8/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5395 - loss: 0.9373 - val_accuracy: 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


113/113 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step
  Model  Precision    Recall  F1-Score  Accuracy
0   CNN   0.475075  0.552778  0.408520  0.552778
1  LSTM   0.452820  0.548889  0.400456  0.548889
2   GRU   0.298541  0.546389  0.386114  0.546389


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Bi-LSTM Modeli
print("Bi-LSTM modeli eğitiliyor...")
bilstm_model = Sequential([
    Embedding(input_dim=len(bert_tokenizer.vocab), output_dim=128, input_length=128),
    Bidirectional(LSTM(64, return_sequences=True)),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(one_hot_labels.shape[1], activation='softmax')
])

bilstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
bilstm_history = bilstm_model.fit(X_train_embeddings, y_train, epochs=10, batch_size=16, validation_data=(X_test_embeddings, y_test))
# Modeller ve Değerlendirme

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd

# Tahminler ve performans metrikleri hesaplama
def evaluate_model_performance(models, X_test, y_test):
    y_true_classes = np.argmax(y_test, axis=1)  # Gerçek sınıflar
    results = []

    for model_name, model in models.items():
        y_pred = model.predict(X_test)  # Model tahmini
        y_pred_classes = np.argmax(y_pred, axis=1)  # Tahmini sınıflar

        # Performans metrikleri hesaplama
        precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
        recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
        f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')
        accuracy = accuracy_score(y_true_classes, y_pred_classes)

        # Sonuçları kaydetme
        results.append({
            "Model": model_name,
            "Precision": precision,
            "Recall": recall,
            "F1-Score": f1,
            "Accuracy": accuracy
        })

    return pd.DataFrame(results)

# Modeller ve değerlendirme
models = {
    "CNN": cnn_model,
    "LSTM": lstm_model,
    "GRU": gru_model,
    "Bi-LSTM": bilstm_model  # Bi-LSTM modelini de dahil ediyoruz
}

# Performans sonuçlarını hesapla ve göster
metrics_df = evaluate_model_performance(models, X_test_embeddings, y_test)
print(metrics_df)

Bi-LSTM modeli eğitiliyor...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 46s 44ms/step - accuracy: 0.5290 - loss: 0.9954 - val_accuracy: 0.5464 - val_loss: 0.9595
Epoch 2/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 41s 45ms/step - accuracy: 0.5323 - loss: 0.9805 - val_accuracy: 0.5464 - val_loss: 0.9589
Epoch 3/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 40s 44ms/step - accuracy: 0.5346 - loss: 0.9753 - val_accuracy: 0.5464 - val_loss: 0.9478
Epoch 4/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 41s 43ms/step - accuracy: 0.5310 - loss: 0.9780 - val_accuracy: 0.5464 - val_loss: 0.9479
Epoch 5/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 41s 44ms/step - accuracy: 0.5359 - loss: 0.9689 - val_accuracy: 0.5464 - val_loss: 0.9453
Epoch 6/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 42s 45ms/step - accuracy: 0.5295 - loss: 0.9751 - val_accuracy: 0.5464 - val_loss: 0.9448
Epoch 7/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 39s 43ms/step - accuracy: 0.5257 - loss: 0.9647 - val_accuracy: 0.5464 - val_loss: 0.9433
Epoch 8/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 40s 44ms/step - accuracy: 0.5326 - loss: 0.9610 - 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step
  8/113 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step
     Model  Precision    Recall  F1-Score  Accuracy
0      CNN   0.475075  0.552778  0.408520  0.552778
1     LSTM   0.452820  0.548889  0.400456  0.548889
2      GRU   0.298541  0.546389  0.386114  0.546389
3  Bi-LSTM   0.436075  0.546944  0.400872  0.546944


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from torch.optim import AdamW

# Cihazı Belirle
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Kullanılan cihaz: {device}")

# Veriyi Yükle
df_cleaned = pd.read_csv('/content/processed_text.csv', encoding='utf-8')
df_original = pd.read_csv('train_0.csv', encoding='utf-8')
df_cleaned['Label'] = df_original['Label']

# One-Hot Encoding
one_hot_encoder = OneHotEncoder(sparse_output=False)
one_hot_labels = one_hot_encoder.fit_transform(df_cleaned['Label'].values.reshape(-1, 1))

# Eğitim ve Test Verisini Ayır
X_train, X_test, y_train, y_test = train_test_split(
    df_cleaned['Processed_Text'],
    one_hot_labels,
    test_size=0.2,
    random_state=42
)

# BERT Tokenizer
bert_tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")

# Tokenizasyon ve Padleme
def tokenize_and_pad(texts, tokenizer, max_len=128):
    tokens = tokenizer(
        texts.tolist(),
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    return tokens['input_ids'], tokens['attention_mask']

X_train_ids, X_train_mask = tokenize_and_pad(X_train, bert_tokenizer)
X_test_ids, X_test_mask = tokenize_and_pad(X_test, bert_tokenizer)

# Tensor Tipi Dönüşümü
y_train = torch.tensor(np.argmax(y_train, axis=1), dtype=torch.long)
y_test = torch.tensor(np.argmax(y_test, axis=1), dtype=torch.long)

# TensorDataset Oluştur
train_dataset = TensorDataset(X_train_ids, X_train_mask, y_train)
test_dataset = TensorDataset(X_test_ids, X_test_mask, y_test)

# DataLoader Oluştur
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# BERT Modeli ve Optimizasyon
bert_model = BertForSequenceClassification.from_pretrained(
    "dbmdz/bert-base-turkish-cased",
    num_labels=len(one_hot_encoder.categories_[0])
).to(device)

optimizer = AdamW(bert_model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Eğitim Döngüsü
def train_model(model, dataloader, optimizer, loss_fn, device):
    model.train()
    total_loss, total_correct = 0, 0
    for batch in dataloader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += (logits.argmax(dim=1) == labels).sum().item()
    return total_loss / len(dataloader), total_correct / len(dataloader.dataset)

# Değerlendirme Döngüsü
def evaluate_model(model, dataloader, loss_fn, device):
    model.eval()
    total_loss, total_correct = 0, 0
    all_labels, all_preds = [], []
    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = loss_fn(logits, labels)

            total_loss += loss.item()
            total_correct += (logits.argmax(dim=1) == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(logits.argmax(dim=1).cpu().numpy())

    report = classification_report(all_labels, all_preds, output_dict=True)
    return total_loss / len(dataloader), total_correct / len(dataloader.dataset), report

# Eğitim ve Değerlendirme Süreci
num_epochs = 5
for epoch in range(num_epochs):
    train_loss, train_accuracy = train_model(bert_model, train_loader, optimizer, loss_fn, device)
    val_loss, val_accuracy, val_report = evaluate_model(bert_model, test_loader, loss_fn, device)
    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    print(f"Validation F1-Score: {val_report['weighted avg']['f1-score']:.4f}")
    print(f"Validation Precision: {val_report['weighted avg']['precision']:.4f}")
    print(f"Validation Recall: {val_report['weighted avg']['recall']:.4f}")

# Test Değerlendirmesi
print("Model test verisi üzerinde değerlendiriliyor...")
test_loss, test_accuracy, test_report = evaluate_model(bert_model, test_loader, loss_fn, device)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")
print(f"Test F1-Score: {test_report['weighted avg']['f1-score']:.4f}")
print(f"Test Precision: {test_report['weighted avg']['precision']:.4f}")
print(f"Test Recall: {test_report['weighted avg']['recall']:.4f}")



Kullanılan cihaz: cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
Train Loss: 0.7546, Train Accuracy: 0.6657
Validation Loss: 0.6383, Validation Accuracy: 0.7289
Validation F1-Score: 0.7214
Validation Precision: 0.7186
Validation Recall: 0.7289
Epoch 2/5
Train Loss: 0.5951, Train Accuracy: 0.7497
Validation Loss: 0.6495, Validation Accuracy: 0.7367
Validation F1-Score: 0.7156
Validation Precision: 0.7269
Validation Recall: 0.7367
Epoch 3/5
Train Loss: 0.4501, Train Accuracy: 0.8125
Validation Loss: 0.6640, Validation Accuracy: 0.7292
Validation F1-Score: 0.7266
Validation Precision: 0.7247
Validation Recall: 0.7292
Epoch 4/5
Train Loss: 0.3051, Train Accuracy: 0.8834
Validation Loss: 0.8009, Validation Accuracy: 0.7111
Validation F1-Score: 0.7110
Validation Precision: 0.7111
Validation Recall: 0.7111
Epoch 5/5
Train Loss: 0.1842, Train Accuracy: 0.9309
Validation Loss: 1.0902, Validation Accuracy: 0.7100
Validation F1-Score: 0.7060
Validation Precision: 0.7046
Validation Recall: 0.7100
Model test verisi üzerinde değerlendiriliyor...
Test Lo

In [ ]:
# Tek Cümle Tahmini Fonksiyonu
def predict_sentence(sentence, tokenizer, model, device):
    model.eval()
    inputs = tokenizer(sentence, max_length=128, padding='max_length', truncation=True, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
        logits = outputs.logits if hasattr(outputs, 'logits') else model(inputs['input_ids'])
    prediction = torch.argmax(logits, dim=1).cpu().item()
    label = one_hot_encoder.categories_[0][prediction]
    return label

# Kullanıcıdan Cümle Al ve Tahmin Yap
user_sentence = input("Bir cümle yazın: ")
predicted_label = predict_sentence(user_sentence, bert_tokenizer, bert_model, device)
print(f"Tahmin Edilen Etiket: {predicted_label}")